## Imports

In [1]:
import io
import os.path
from scrapy import Selector
from datetime import datetime
from urllib.request import Request, urlopen

## Baixando HTML do site Fundamentus (Web Crawling)

In [2]:
# mudar o nome do papel para baixar novo HTML
papel = 'GGBR4'
url = 'https://fundamentus.com.br/detalhes.php?papel=%s' % papel

req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})

response = urlopen(req, timeout=20).read()
html_dados = response.decode('latin-1')

In [ ]:
sel = Selector( text = html_dados )

## Pegando nome do papel, data última cotação e data último balanço

In [16]:
papel = sel.xpath("//table[1]//span[@class='txt']/text()").extract()[1]
papel

'GGBR4'

In [17]:
data_ult_cotacao = sel.xpath("//table[1]//span[@class='txt']/text() | //table[1]//span[@class='txt']/a/text()").extract()[7]
data_ult_cotacao = datetime.strptime(data_ult_cotacao, '%d/%m/%Y').date()
data_ult_cotacao = str(data_ult_cotacao)
data_ult_cotacao

'2021-03-01'

In [18]:
data_ult_balanco = sel.xpath("//table[2]//span[@class='txt']/text()").extract()[3]
data_ult_balanco = datetime.strptime(data_ult_balanco, '%d/%m/%Y').date()
data_ult_balanco = str(data_ult_balanco)
data_ult_balanco

'2020-12-31'

## Funções para transformar os dados e labels

In [94]:
from unicodedata import normalize
def remover_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

if __name__ == '__main__':
    from doctest import testmod
    testmod()

In [95]:
def remover_caracteres_especiais(empresa_label):
    empresa_label = list(map(lambda x: remover_acentos(x), empresa_label))
    empresa_label = list(map(lambda st: str.replace(st, " ","_"), empresa_label))
    empresa_label = list(map(lambda st: str.replace(st, "(",""), empresa_label))
    empresa_label = list(map(lambda st: str.replace(st, ")",""), empresa_label))
    return empresa_label

In [96]:
def transformar_lista_em_tuples(empresa_dados):
    empresa_dados_list = []
    empresa_dados_list.append(tuple(empresa_dados))
    return empresa_dados_list

## Verificação se já existe os dados:
* Verificar se o papel já existe nos dados gravados
    * Se não existir, inserir dados da empresa, dados de cotação e dados do balanço
    * Se existir, verificar se a cotação mais atual já existe
    * Se existir, verificar se o balanço mais atual já existe
* Verificando se a cotaçao mais atual já existe
    * Se não existir, inserir dados da cotação atualizado
    * Se existir, não fazer nada
* Verificando se o balanço mais atual já existe
    * Se não existir, inserir dados do balanço atualizado
    * Se existir, não fazer nada

## Incluindo Dados da Empresa (caso ainda não exista)

In [97]:
info_empresa_label = sel.xpath("//table[1]//span[@class='txt']/text() | //table[1]//span[@class='txt']/a/text()").extract()[0::2][0::2]
info_empresa_label = remover_caracteres_especiais(info_empresa_label)
info_empresa_label

['Papel', 'Tipo', 'Empresa', 'Setor', 'Subsetor']

In [26]:
info_empresa_dados = sel.xpath("//table[1]//span[@class='txt']/text() | //table[1]//span[@class='txt']/a/text()").extract()[1::2][0::2]
info_empresa_dados

['GGBR4',
 'PN N1',
 'GERDAU S.A. PN N1',
 'Siderurgia e Metalurgia',
 'Siderurgia']

## Inclundo Dados atualizados de Cotação (caso estejam desatualizados)
### Dados Básicos Cotação

In [60]:
cotacao_label = sel.xpath("//table[1]//span[@class='txt']/text() | //table[1]//span[@class='txt']/a/text()").extract()[0::2][1::2]
cotacao_label

['Cotação', 'Data últ cot', 'Min 52 sem', 'Max 52 sem', 'Vol $ méd (2m)']

In [61]:
cotacao_dados = sel.xpath("//table[1]//span[@class='txt']/text() | //table[1]//span[@class='txt']/a/text()").extract()[1::2][1::2]
cotacao_dados

['26,67', '01/03/2021', '8,54', '29,54', '419.645.000']

In [98]:
cotacao_label += sel.xpath("//table[2]//span[@class='txt']/text()").extract()[0::2][0::3]
cotacao_label = remover_caracteres_especiais(cotacao_label)
cotacao_label

['Cotacao',
 'Data_ult_cot',
 'Min_52_sem',
 'Max_52_sem',
 'Vol_$_med_2m',
 'Valor_de_mercado',
 'Nro._Acoes',
 'Valor_de_mercado',
 'Nro._Acoes']

In [63]:
cotacao_dados += sel.xpath("//table[2]//span[@class='txt']/text()").extract()[1::2][0::3]
cotacao_dados

['26,67',
 '01/03/2021',
 '8,54',
 '29,54',
 '419.645.000',
 '45.863.300.000',
 '1.719.660.000']

### Dados Oscilação

In [64]:
oscilacao_label = sel.xpath("//table[3]//td[@class='nivel1']//span/text()").extract()[0]
oscilacao_label

'Oscilações'

In [66]:
lista_oscilacoes_label = sel.xpath("//table[3]//td[@class='label w1']//span/text()").extract()
lista_oscilacoes_label

['Dia',
 'Mês',
 '30 dias',
 '12 meses',
 '2021',
 '2020',
 '2019',
 '2018',
 '2017',
 '2016']

In [99]:
oscilacoes_label = list(map(lambda label: oscilacao_label + "_" + label, lista_oscilacoes_label))
oscilacoes_label = remover_caracteres_especiais(oscilacoes_label)
oscilacoes_label

['Oscilacoes_Dia',
 'Oscilacoes_Mes',
 'Oscilacoes_30_dias',
 'Oscilacoes_12_meses',
 'Oscilacoes_2021',
 'Oscilacoes_2020',
 'Oscilacoes_2019',
 'Oscilacoes_2018',
 'Oscilacoes_2017',
 'Oscilacoes_2016']

In [69]:
oscilacoes_dados = sel.xpath("//table[3]//span[@class='oscil']/font/text()").extract()
oscilacoes_dados

['3,09%',
 '3,09%',
 '16,11%',
 '55,83%',
 '9,08%',
 '23,86%',
 '37,65%',
 '22,31%',
 '15,23%',
 '138,66%']

### Dados Indicadores Fundamentalistas

In [100]:
indicador_fundamentalita_label = sel.xpath("//table[3]//td[@class='label w2']//span[@class='txt']/text() | //table[3]//td[@class='label']//span[@class='txt']/text()").extract()
indicador_fundamentalita_label = remover_caracteres_especiais(indicador_fundamentalita_label)
indicador_fundamentalita_label

['P/L',
 'LPA',
 'P/VP',
 'VPA',
 'P/EBIT',
 'Marg._Bruta',
 'PSR',
 'Marg._EBIT',
 'P/Ativos',
 'Marg._Liquida',
 'P/Cap._Giro',
 'EBIT_/_Ativo',
 'P/Ativ_Circ_Liq',
 'ROIC',
 'Div._Yield',
 'ROE',
 'EV_/_EBITDA',
 'Liquidez_Corr',
 'EV_/_EBIT',
 'Div_Br/_Patrim',
 'Cres._Rec_5a',
 'Giro_Ativos']

In [72]:
indicador_fundamentalita_dados = sel.xpath("//table[3]//td[@class='data w2']//span[@class='txt']/text() | //table[3]//td[@class='data']//span[@class='txt']/text()").extract()
indicador_fundamentalita_dados

['19,39',
 '1,38',
 '1,49',
 '17,95',
 '\n10,42',
 '\n13,5%',
 '\n1,05',
 '\n10,0%',
 '\n0,73',
 '\n5,5%',
 '\n3,85',
 '7,0%',
 '\n-5,32',
 '\n8,8%',
 '1,1%',
 '\n7,7%',
 '\n8,14',
 '\n2,04',
 '\n12,66',
 '\n0,57',
 '\n3,8%',
 '\n0,69']

## Incluindo Dados atualizados do Balanço (caso estejam desatualizados)

In [76]:
balanco_label = sel.xpath("//table[2]//span[@class='txt']/text()").extract()[2:6][0::2]
balanco_label

['Últ balanço processado', 'Valor da firma']

In [83]:
balanco_dados = sel.xpath("//table[2]//span[@class='txt']/text()").extract()[2:6][1::2]
balanco_dados

['31/12/2020', '55.720.300.000']

In [81]:
balanco_label += sel.xpath("//table[4]//td[@class='label w2']//span[@class='txt']/text() | //table[4]//td[@class='label']//span[@class='txt']/text()").extract()
balanco_label

['Últ balanço processado',
 'Valor da firma',
 'Ativo',
 'Dív. Bruta',
 'Disponibilidades',
 'Dív. Líquida',
 'Ativo Circulante',
 'Patrim. Líq']

In [84]:
balanco_dados += sel.xpath("//table[4]//td[@class='data w3']//span[@class='txt']/text() | //table[4]//td[@class='data']//span[@class='txt']/text()").extract()
balanco_dados

['31/12/2020',
 '55.720.300.000',
 '63.123.000.000',
 '17.515.400.000',
 '7.658.350.000',
 '9.857.000.000',
 '23.409.500.000',
 '30.860.300.000']

In [87]:
dre_12_meses = sel.xpath("//table[5]//td[@class='label w2']//span[@class='txt']/text() | //table[5]//td[@class='label']//span[@class='txt']/text()").extract()[0::2]
balanco_label += list(map(lambda label: label + "_ult._12_meses", dre_12_meses))
balanco_label

['Últ balanço processado',
 'Valor da firma',
 'Ativo',
 'Dív. Bruta',
 'Disponibilidades',
 'Dív. Líquida',
 'Ativo Circulante',
 'Patrim. Líq',
 'Receita Líquida_ult._12_meses',
 'EBIT_ult._12_meses',
 'Lucro Líquido_ult._12_meses']

In [90]:
balanco_dados += sel.xpath("//table[5]//td[@class='data w3']//span[@class='txt']/text() | //table[5]//td[@class='data']//span[@class='txt']/text()").extract()[0::2]
balanco_dados

['31/12/2020',
 '55.720.300.000',
 '63.123.000.000',
 '17.515.400.000',
 '7.658.350.000',
 '9.857.000.000',
 '23.409.500.000',
 '30.860.300.000',
 '43.814.700.000',
 '4.400.170.000',
 '2.365.760.000']

In [101]:
dre_3_meses = sel.xpath("//table[5]//td[@class='label w2']//span[@class='txt']/text() | //table[5]//td[@class='label']//span[@class='txt']/text()").extract()[1::2]
balanco_label += list(map(lambda label: label + "_ult._3_meses", dre_3_meses))
balanco_label

['Ult_balanco_processado',
 'Valor_da_firma',
 'Ativo',
 'Div._Bruta',
 'Disponibilidades',
 'Div._Liquida',
 'Ativo_Circulante',
 'Patrim._Liq',
 'Receita_Liquida_ult._12_meses',
 'EBIT_ult._12_meses',
 'Lucro_Liquido_ult._12_meses',
 'Receita_Liquida_ult._3_meses',
 'EBIT_ult._3_meses',
 'Lucro_Liquido_ult._3_meses',
 'Receita_Liquida_ult._3_meses',
 'EBIT_ult._3_meses',
 'Lucro_Liquido_ult._3_meses']

In [ ]:
balanco_label = remover_caracteres_especiais(balanco_label)
balanco_label

In [93]:
balanco_dados += sel.xpath("//table[5]//td[@class='data w3']//span[@class='txt']/text() | //table[5]//td[@class='data']//span[@class='txt']/text()").extract()[1::2]
balanco_dados

['31/12/2020',
 '55.720.300.000',
 '63.123.000.000',
 '17.515.400.000',
 '7.658.350.000',
 '9.857.000.000',
 '23.409.500.000',
 '30.860.300.000',
 '43.814.700.000',
 '4.400.170.000',
 '2.365.760.000',
 '13.620.200.000',
 '2.176.850.000',
 '1.048.330.000']

# -----------------------------------------------------------------------------------------------

## Verificando se o arquivo já existe

In [5]:
path = '/home/hugo/Documents/Repositorios_GitHub/Projetos/02-StockData/Web_Scraping/Fundamentus_Web/'
file_name = '%s_fundamentus_%s.txt' % (papel, str(data_ult_balanco_new_html))

if os.path.isfile(path+file_name):
    print('Arquivo já existe com balanço mais atual!')
else:
    print('Arquivo com balanço atual ainda não existe!')

Arquivo já existe com balanço mais atual!


## Salvando conteúdo do HTML (em bytes) em um arquivo local como txt

In [6]:
file_name = '%s_fundamentus_%s.txt' % (papel, data_ult_balanco_new_html)

with io.open(path+file_name, 'w', encoding='utf-8') as f:
    f.write(new_html)

## Lendo conteúdo do arquivo Local e armazenando em uma variável

In [7]:
with io.open(path+file_name, 'r', encoding='utf8') as f:
    html = f.read()

# ------------------------------------------------------------------------------------------------

## Extrair e Separar Labels dos Dados do HTML (Web Scraping)
### Tabela 1 e 2: Dados da Empresa e última cotação

In [10]:
empresa_cotacao = sel.xpath("//table[1]//span[@class='txt']/text() | //table[1]//span[@class='txt']/a/text()").extract()
valores_data_balanco = sel.xpath("//table[2]//span[@class='txt']/text()").extract()

In [17]:
empresa_label = empresa_cotacao[0::2] + valores_data_balanco[0::2]
empresa_label = remover_caracteres_especiais(empresa_label)
empresa_label

['Papel',
 'Cotacao',
 'Tipo',
 'Data_ult_cot',
 'Empresa',
 'Min_52_sem',
 'Setor',
 'Max_52_sem',
 'Subsetor',
 'Vol_$_med_2m',
 'Valor_de_mercado',
 'Ult_balanco_processado',
 'Valor_da_firma',
 'Nro._Acoes']

In [22]:
empresa_dados = empresa_cotacao[1::2] + valores_data_balanco[1::2]
empresa_dados = transformar_lista_em_tuples(empresa_dados)
empresa_dados

[('GGBR4',
  '26,67',
  'PN N1',
  '01/03/2021',
  'GERDAU S.A. PN N1',
  '8,54',
  'Siderurgia e Metalurgia',
  '29,54',
  'Siderurgia',
  '419.645.000',
  '45.863.300.000',
  '31/12/2020',
  '55.720.300.000',
  '1.719.660.000')]

In [24]:
papel = empresa_dados[0][0]
papel

'GGBR4'

### Tabela 3.1: Oscilações

In [13]:
sel.xpath("//table[3]//td[@class='nivel1']//span/text()").extract()[0]

'Oscilações'

In [25]:
sel.xpath("//table[3]//td[@class='nivel1']//span/text()").extract()[0]
oscilacoes_label = sel.xpath("//table[3]//td[@class='label w1']//span/text()").extract()
oscilacoes_label.insert(0, 'Papel')
oscilacoes_label = remover_caracteres_especiais(oscilacoes_label)
oscilacoes_label

['Papel',
 'Dia',
 'Mes',
 '30_dias',
 '12_meses',
 '2021',
 '2020',
 '2019',
 '2018',
 '2017',
 '2016']

In [26]:
oscilacoes_dados = sel.xpath("//table[3]//span[@class='oscil']/font/text()").extract()
oscilacoes_dados.insert(0, papel)
oscilacoes_dados = transformar_lista_em_tuples(oscilacoes_dados)
oscilacoes_dados

[('GGBR4',
  '3,09%',
  '3,09%',
  '14,76%',
  '59,60%',
  '9,08%',
  '23,86%',
  '37,65%',
  '22,31%',
  '15,23%',
  '138,66%')]

### Tabela 3.2: Indicadores fundamentalistas

In [16]:
sel.xpath("//table[3]//td[@class='nivel1']//span/text()").extract()[1]

'Indicadores fundamentalistas'

In [27]:
indicadores_label = sel.xpath("//table[3]//td[@class='label w2']//span[@class='txt']/text() | //table[3]//td[@class='label']//span[@class='txt']/text()").extract()
indicadores_label.insert(0, 'Papel')
indicadores_label = remover_caracteres_especiais(indicadores_label)
indicadores_label

['Papel',
 'P/L',
 'LPA',
 'P/VP',
 'VPA',
 'P/EBIT',
 'Marg._Bruta',
 'PSR',
 'Marg._EBIT',
 'P/Ativos',
 'Marg._Liquida',
 'P/Cap._Giro',
 'EBIT_/_Ativo',
 'P/Ativ_Circ_Liq',
 'ROIC',
 'Div._Yield',
 'ROE',
 'EV_/_EBITDA',
 'Liquidez_Corr',
 'EV_/_EBIT',
 'Div_Br/_Patrim',
 'Cres._Rec_5a',
 'Giro_Ativos']

In [31]:
# Os dado estão vindo irregulares, com \n na frente de alguns valores
indicadores_dados = sel.xpath("//table[3]//td[@class='data w2']//span[@class='txt']/text() | //table[3]//td[@class='data']//span[@class='txt']/text()").extract()
indicadores_dados.insert(0, papel)
indicadores_dados = list(map(lambda st: str.replace(st, "\n",""), indicadores_dados))
indicadores_dados = transformar_lista_em_tuples(indicadores_dados)
indicadores_dados

[('GGBR4',
  '19,39',
  '1,38',
  '1,49',
  '17,95',
  '10,42',
  '13,5%',
  '1,05',
  '10,0%',
  '0,73',
  '5,5%',
  '3,85',
  '7,0%',
  '-5,32',
  '8,8%',
  '1,1%',
  '7,7%',
  '8,14',
  '2,04',
  '12,66',
  '0,57',
  '3,8%',
  '0,69')]

### Tabela 4: Dados Balanço Patrimonial

In [32]:
sel.xpath("//table[4]//td[@class='nivel1']//span/text()").extract()[0]

'Dados Balanço Patrimonial'

In [33]:
balanco_patrimonial_label = sel.xpath("//table[4]//td[@class='label w2']//span[@class='txt']/text() | //table[4]//td[@class='label']//span[@class='txt']/text()").extract()
balanco_patrimonial_label.insert(0, 'Papel')
balanco_patrimonial_label = remover_caracteres_especiais(balanco_patrimonial_label)
balanco_patrimonial_label

['Papel',
 'Ativo',
 'Div._Bruta',
 'Disponibilidades',
 'Div._Liquida',
 'Ativo_Circulante',
 'Patrim._Liq']

In [34]:
balanco_patrimonial_dados = sel.xpath("//table[4]//td[@class='data w3']//span[@class='txt']/text() | //table[4]//td[@class='data']//span[@class='txt']/text()").extract()
balanco_patrimonial_dados.insert(0, papel)
balanco_patrimonial_dados = transformar_lista_em_tuples(balanco_patrimonial_dados)
balanco_patrimonial_dados

[('GGBR4',
  '63.123.000.000',
  '17.515.400.000',
  '7.658.350.000',
  '9.857.000.000',
  '23.409.500.000',
  '30.860.300.000')]

### Tabela 5.1: Demonstrativos de Resultados - Últimos 12 meses

In [194]:
sel.xpath("//table[5]//td[@class='nivel1']//span/text()").extract()[0]

'Dados demonstrativos de resultados'

In [62]:
label_ult_12_meses = sel.xpath("//table[5]//td[@class='label w2']//span[@class='txt']/text() | //table[5]//td[@class='label']//span[@class='txt']/text()").extract()[0::2]
label_ult_12_meses = list(map(lambda label: label + '_ult_12_meses', label_ult_12_meses))
demonstrativo_resultados_label = label_ult_12_meses

In [63]:
label_ult_3_meses = sel.xpath("//table[5]//td[@class='label w2']//span[@class='txt']/text() | //table[5]//td[@class='label']//span[@class='txt']/text()").extract()[1::2]
label_ult_3_meses = list(map(lambda label: label + '_ult_3_meses', label_ult_3_meses))
demonstrativo_resultados_label += label_ult_3_meses

In [64]:
demonstrativo_resultados_label.insert(0, 'Papel')
demonstrativo_resultados_label = remover_caracteres_especiais(demonstrativo_resultados_label)
demonstrativo_resultados_label

['Papel',
 'Receita_Liquida_ult_12_meses',
 'EBIT_ult_12_meses',
 'Lucro_Liquido_ult_12_meses',
 'Receita_Liquida_ult_3_meses',
 'EBIT_ult_3_meses',
 'Lucro_Liquido_ult_3_meses']

In [67]:
dados_ult_12_meses = sel.xpath("//table[5]//td[@class='data w3']//span[@class='txt']/text() | //table[5]//td[@class='data']//span[@class='txt']/text()").extract()[0::2]
dados_ult_3_meses = sel.xpath("//table[5]//td[@class='data w3']//span[@class='txt']/text() | //table[5]//td[@class='data']//span[@class='txt']/text()").extract()[1::2]
demonstrativo_resultados_dados = dados_ult_12_meses + dados_ult_3_meses
demonstrativo_resultados_dados.insert(0, papel)
demonstrativo_resultados_dados = transformar_lista_em_tuples(demonstrativo_resultados_dados)
demonstrativo_resultados_dados

[('GGBR4',
  '43.814.700.000',
  '4.400.170.000',
  '2.365.760.000',
  '13.620.200.000',
  '2.176.850.000',
  '1.048.330.000')]

# Acesso ao HDFS server (Write and Read)

In [162]:
#!pip3 install pyspark

In [163]:
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.appName("example-pyspark-read-and-write").getOrCreate()

In [164]:
sparkSession.createDataFrame(empresa_dados_list).collect()

[Row(_1='GGBR4', _2='25,87', _3='PN N1', _4='26/02/2021', _5='GERDAU S.A. PN N1', _6='8,54', _7='Siderurgia e Metalurgia', _8='29,54', _9='Siderurgia', _10='402.679.000', _11='44.487.600.000', _12='31/12/2020', _13='54.344.600.000', _14='1.719.660.000')]

In [175]:
sparkSession.createDataFrame(empresa_dados_list, empresa_label).collect()

[Row(Papel='GGBR4', Cotacao='25,87', Tipo='PN N1', Data_ult_cot='26/02/2021', Empresa='GERDAU S.A. PN N1', Min_52_sem='8,54', Setor='Siderurgia e Metalurgia', Max_52_sem='29,54', Subsetor='Siderurgia', Vol_$_med_2m='402.679.000', Valor_de_mercado='44.487.600.000', Ult_balanco_processado='31/12/2020', Valor_da_firma='54.344.600.000', Nro._Acoes='1.719.660.000')]

In [176]:
df = sparkSession.createDataFrame(empresa_dados_list, empresa_label)
df

DataFrame[Papel: string, Cotacao: string, Tipo: string, Data_ult_cot: string, Empresa: string, Min_52_sem: string, Setor: string, Max_52_sem: string, Subsetor: string, Vol_$_med_2m: string, Valor_de_mercado: string, Ult_balanco_processado: string, Valor_da_firma: string, Nro._Acoes: string]

In [177]:
type(df)

pyspark.sql.dataframe.DataFrame

In [178]:
df.show()

+-----+-------+-----+------------+-----------------+----------+--------------------+----------+----------+------------+----------------+----------------------+--------------+-------------+
|Papel|Cotacao| Tipo|Data_ult_cot|          Empresa|Min_52_sem|               Setor|Max_52_sem|  Subsetor|Vol_$_med_2m|Valor_de_mercado|Ult_balanco_processado|Valor_da_firma|   Nro._Acoes|
+-----+-------+-----+------------+-----------------+----------+--------------------+----------+----------+------------+----------------+----------------------+--------------+-------------+
|GGBR4|  25,87|PN N1|  26/02/2021|GERDAU S.A. PN N1|      8,54|Siderurgia e Meta...|     29,54|Siderurgia| 402.679.000|  44.487.600.000|            31/12/2020|54.344.600.000|1.719.660.000|
+-----+-------+-----+------------+-----------------+----------+--------------------+----------+----------+------------+----------------+----------------------+--------------+-------------+



### Necessário configurar a versão do Python utilizada pelo Linux como a 3.6.
* sudo update-alternatives --install /usr/bin/python python /usr/bin/python3.4 1
* sudo update-alternatives --install /usr/bin/python python /usr/bin/python3.6 2
* sudo update-alternatives --config python
* sudo update-alternatives  --set python /usr/bin/python3.6

In [179]:
# Write into HDFS
# hdfs://hadoop-master:9000/user/hadoopuser/test/
df.write.save('hdfs://172.17.177.40:9000/user/hadoopuser/fundamentus/detalhes/dados_empresa', format='parquet', mode='append')

In [180]:
df_load = sparkSession.read.format('parquet').load('hdfs://172.17.177.40:9000/user/hadoopuser/fundamentus/detalhes/dados_empresa')

In [181]:
df_load.show()

+-----+-------+-----+------------+-----------------+----------+--------------------+----------+----------+------------+----------------+----------------------+--------------+-------------+
|Papel|Cotacao| Tipo|Data_ult_cot|          Empresa|Min_52_sem|               Setor|Max_52_sem|  Subsetor|Vol_$_med_2m|Valor_de_mercado|Ult_balanco_processado|Valor_da_firma|   Nro._Acoes|
+-----+-------+-----+------------+-----------------+----------+--------------------+----------+----------+------------+----------------+----------------------+--------------+-------------+
|GGBR4|  25,87|PN N1|  26/02/2021|GERDAU S.A. PN N1|      8,54|Siderurgia e Meta...|     29,54|Siderurgia| 402.679.000|  44.487.600.000|            31/12/2020|54.344.600.000|1.719.660.000|
+-----+-------+-----+------------+-----------------+----------+--------------------+----------+----------+------------+----------------+----------------------+--------------+-------------+



In [182]:
df_load.toPandas()

,Papel,Cotacao,Tipo,Data_ult_cot,Empresa,Min_52_sem,Setor,Max_52_sem,Subsetor,Vol_$_med_2m,Valor_de_mercado,Ult_balanco_processado,Valor_da_firma,Nro._Acoes
0,GGBR4,"25,87",PN N1,26/02/2021,GERDAU S.A. PN N1,"8,54",Siderurgia e Metalurgia,"29,54",Siderurgia,402.679.000,44.487.600.000,31/12/2020,54.344.600.000,1.719.660.000
